In [ ]:
import import_ipynb
import cnn_genre_classifier as mgr
import numpy as np
import torch
from pytorch_model_summary import summary

# Data preparation

In [ ]:
# create new sets from extracted spectrogram/mfcc features file or load existing sets

# DATA_PATH = "/data/shared/GTZAN-DATASET/spectrograms_10segments.npy"
# DATA_PATH = "/data/shared/GTZAN-DATASET/mfccs40_10segments.npy"
# X_train, X_validation, X_test, y_train, y_validation, y_test, scale_min, scale_max = mgr.load_data(DATA_PATH, test_size=0.20, validation_size=0.15, scale=True)

feature_sets = torch.load("./data/feature_sets/spectro10_normalized.pth")
# feature_sets = torch.load("./data/feature_sets/40mfcc10_normalized.pth")
X_train = feature_sets['X_train']
X_validation = feature_sets['X_validation']
X_test = feature_sets['X_test']
y_train = feature_sets['y_train']
y_validation = feature_sets['y_validation']
y_test = feature_sets['y_test']
scale_min = feature_sets['scale_min']
scale_max = feature_sets['scale_max']

mgr.plot_feature_sets(y_train, y_validation, y_test)

# Training and results

In [ ]:
# load existing or create new model

model = mgr.new_model()
# model = mgr.load_model("./data/saved_models/model-spectro10-epoch15-batch64_512-norm.pth")
# history = torch.load("./data/saved_models/model-spectro10-epoch15-batch64_512-norm_history.pth")

print(summary(model, torch.rand(1, 1, np.shape(X_train)[2], np.shape(X_train)[3]).cuda()), sep='\n')

In [ ]:
# training loop

history = model.fit(
    X_train, y_train,
    validation_data=(X_validation, y_validation),
    epochs=5,
    batch_size=64,
    log=True,
    history=None)

In [ ]:
# plot results

mgr.plot_history(history)
print("Max_train_acc:", max(history['acc']), "  Min_train_loss:", min(history['loss']))
print("Max_val_acc:", max(history['val_acc']), "  Min_val_loss:", min(history['val_loss']))
t_acc, t_loss = model.test(X_test, y_test, out=True)

In [ ]:
# create and plot confusion matrix

model_predictions = model.get_predictions(X_test)
matrix = mgr.create_confusion_matrix(model_predictions, y_test)
mgr.plot_confusion_matrix(matrix)

matrix = matrix.numpy()
true_pos = np.diag(matrix)
false_pos = np.sum(matrix, axis=0) - true_pos
false_neg = np.sum(matrix, axis=1) - true_pos

precision = true_pos / (true_pos + false_pos)
recall = true_pos / (true_pos + false_neg)

In [ ]:
# rename model

model.model_name = 'model-spectro10-epoch15-batch64_512-norm'

In [ ]:
# save model

mgr.save_model(model, "./data/saved_models")

In [ ]:
# save training history

torch.save(history, "./data/saved_models/"  + model.model_name + "_history.pth")